<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/analysis/293T/obs2/filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sbooeshaghi/azucar.git

Cloning into 'azucar'...
remote: Enumerating objects: 1470, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 1470 (delta 129), reused 24 (delta 3), pack-reused 1242
Receiving objects: 100% (1470/1470), 1.70 GiB | 30.62 MiB/s, done.
Resolving deltas: 100% (605/605), done.
Checking out files: 100% (279/279), done.


In [2]:
#@title import
import os
import matplotlib.pyplot as plt
from sklearn.metrics import rand_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
import json
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from collections import defaultdict
from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix
from sklearn.neighbors import KDTree
from scipy.stats import entropy
from itertools import combinations
import sys
import gzip
from scipy.stats import entropy
from sklearn.mixture import GaussianMixture


def nd(arr):
    return np.asarray(arr).reshape(-1)

def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [12]:
#@title mx filter

def read_str_list(fname, lst=list):
  with open(fname, 'r') as f:
    for idx, line in enumerate(f.readlines()):
      lst.append(line.strip())

def write_list(fname, lst=list):
  with open(fname, 'w') as f:
    for idx, ele in enumerate(lst):
      f.write(f"{ele}\n")

def knee(mtx, axis):
    u = nd(mtx.sum(axis)) # counts per barcode
    x = np.sort(u)[::-1] # sorted
    v = np.log1p(x).reshape(-1, 1) # log1p and reshaped for gmm
    return (u, x, v)

def gmm(x, v, comps):#, n_comps = 2, n_iter=1):
    n_comps = comps.pop(0)

    gm = GaussianMixture(n_components=n_comps, random_state=42)
    labels = gm.fit_predict(v)
    prob = gm.predict_proba(v)
    ent = entropy(prob, axis=1)
    
    # index of v where low count cell is
    if n_comps == 2:
        ind = np.argmax(ent)
        log1p_cutoff = v[ind][0]
        cutoff = x[ind]
    elif n_comps > 2:
        # sort means, and pick the range of the top two
        means = np.sort((np.exp(gm.means_)-1).flatten())
        r = np.logical_and(x>means[-2], x<means[-1]) # make ranage
        df = pd.DataFrame({"ent": ent, "idx": np.arange(ent.shape[0]).astype(int)})[r]
        amax = df["ent"].argmax() # get the index (of x) where the entropy is the max (in range r)
        idx = df.iloc[amax]["idx"].astype(int)
        cutoff = x[idx]
    
    # n_iter -= 1
    n_iter = len(comps)
    if n_iter <= 0:
        return (cutoff, (x>cutoff).sum())
    return gmm(x[x>cutoff], v[x>cutoff], comps)#, n_comps, n_iter)


def mx_filter(matrix_fn, md_fn, matrix_fn_out, md_fn_out, axis=1, comps=[2]):
    # read matrix
    mtx  = mmread(matrix_fn).tocsr()

    # read barcodes
    md = []
    read_str_list(md_fn, md)
#     cmask = nd(mtx.sum(1) > 2)
#     mtx = mtx[cmask]
#     barcodes = np.array(barcodes)[cmask]

    # find knee
    # check this, do it twice?
    u, x, v = knee(mtx, axis)
    (cutoff, ncells) = gmm(x, v, comps=comps) #n_iter=n_iter, n_comps=n_comps)
    # (cutoff, ncells) = gmm(x[x>cutoff], v[x>cutoff], n_iter=1, n_comps=3)

    print(
        f"Filtered to {ncells:,.0f} cells with at least {cutoff:,.0f} UMIs."
    )

    # mask matrix and netadata
    mask = u > cutoff
    mtx_f = mtx[mask]
    md_f = np.array(md)[mask]
    
    # save filtered matrix
    mmwrite(matrix_fn_out, mtx_f)
    
    # save filtered metadata
    write_list(md_fn_out, md_f)

In [13]:
sample = "293T"
observation = "obs4"

base_data = f"azucar/analysis/{sample}/{observation}/out"
base_mark = f"azucar/analysis/{sample}/{observation}/assign"

matrix_fn  = os.path.join(base_data, "matrix.mtx")
genes_fn   = os.path.join(base_data, "genes.txt")
barcodes_fn   = os.path.join(base_data, "barcodes.txt")

!gunzip $base_data/*.gz

gzip: azucar/analysis/293T/obs4/out/*.gz: No such file or directory


In [14]:
# knee plot gmm filter
mx_filter(matrix_fn,
          barcodes_fn,
          "./san.matrix.mtx", 
          "./san.barcodes.txt",
          comps=[2])

Filtered to 79,456 cells with at least 9 UMIs.
